In [12]:
import requests
import json
from pprint import pprint
import pandas as pd
from mapquestkey import key

In [4]:
# Read the dataset cleaned
csv_path = "Resources/jailbook_cleaned.csv"                    
df = pd.read_csv(csv_path)
df.head()

,Unnamed: 0,Defendant,Address,Zip,City,State,Booking_Date,Booking_year_month,Date_of_birth,Age,day_of_week,Crime_Family,Crime_Family1,Crime_Family2,CityRN
0,1,"DITAS, MICHAEL",7484 NW 169 TERR,NaN,MIAMI,FL,2015-06-05,2015-06,1982-02-17,37,Friday,"THEFT, ROBBERY, AND RELATED CRIMES",ASSAULT & BATTERY,PERSONAL CRIME,MIAMI
1,4,"DORSEY, JOHN",651 NW 177TH STREET 101,NaN,MIAMI GARDENS,FL,2015-06-05,2015-06,1985-04-11,34,Friday,ALCOHOL PROHIBITION,DRUGS & ALCOHOL,STATUTORY CRIME,MIAMI
2,16,"GAITOR, RASHAD ZAROSHUA",UNKNOWN,NaN,MIAMI,FL,2015-06-05,2015-06,1991-08-19,27,Friday,WARRANT,MISCELLANEOUS CRIMES,OTHER,MIAMI
3,24,"WILLIAMS, ALAN J",UNKNOWN,NaN,MIAMI,FL,2015-06-03,2015-06,1964-04-28,55,Wednesday,ALCOHOL PROHIBITION,DRUGS & ALCOHOL,STATUTORY CRIME,MIAMI
4,29,"WYNTER, KALONJI ABAYOMI",HOMELESS,NaN,HOMELESS,FL,2015-06-03,2015-06,1977-05-19,42,Wednesday,WARRANT,MISCELLANEOUS CRIMES,OTHER,HOMELESS


In [5]:
# # URL for GET requests to retrieve mapquest data
url = "http://www.mapquestapi.com/geocoding/v1/address?key=IYNehGBC4T9Qt5APqoIcLN7LfJWr7arr&location=5536%20NW%2021ST%20AVENUE%20MIAMI,%20FL%2033142"

# # Print the response object to the console
print(requests.get(url))

# # Retrieving data and converting it into JSON
print(requests.get(url).json())

# # Pretty Print the output of the JSON
response = requests.get(url).json()
pprint(json.dumps(response, indent=4, sort_keys=True))

<Response [200]>
{'info': {'statuscode': 0, 'copyright': {'text': '© 2019 MapQuest, Inc.', 'imageUrl': 'http://api.mqcdn.com/res/mqlogo.gif', 'imageAltText': '© 2019 MapQuest, Inc.'}, 'messages': []}, 'options': {'maxResults': -1, 'thumbMaps': True, 'ignoreLatLngInput': False}, 'results': [{'providedLocation': {'location': '5536 NW 21ST AVENUE MIAMI, FL 33142'}, 'locations': [{'street': '5536 NW 21st Ave', 'adminArea6': '', 'adminArea6Type': 'Neighborhood', 'adminArea5': 'Miami', 'adminArea5Type': 'City', 'adminArea4': 'Miami-Dade', 'adminArea4Type': 'County', 'adminArea3': 'FL', 'adminArea3Type': 'State', 'adminArea1': 'US', 'adminArea1Type': 'Country', 'postalCode': '33142-3030', 'geocodeQualityCode': 'P1AAA', 'geocodeQuality': 'POINT', 'dragPoint': False, 'sideOfStreet': 'L', 'linkId': 'r12598819|p31271369|n15725758', 'unknownInput': '', 'type': 's', 'latLng': {'lat': 25.825129, 'lng': -80.230688}, 'displayLatLng': {'lat': 25.824858, 'lng': -80.231591}, 'mapUrl': 'http://www.mapques

In [6]:
# # set up additional columns to hold information
df['lat'] = ""
df['lon'] = ""
df['Zip_code_from_api'] = ""

In [7]:
df.head()

,Unnamed: 0,Defendant,Address,Zip,City,State,Booking_Date,Booking_year_month,Date_of_birth,Age,day_of_week,Crime_Family,Crime_Family1,Crime_Family2,CityRN,lat,lon,Zip_code_from_api
0,1,"DITAS, MICHAEL",7484 NW 169 TERR,NaN,MIAMI,FL,2015-06-05,2015-06,1982-02-17,37,Friday,"THEFT, ROBBERY, AND RELATED CRIMES",ASSAULT & BATTERY,PERSONAL CRIME,MIAMI,,,
1,4,"DORSEY, JOHN",651 NW 177TH STREET 101,NaN,MIAMI GARDENS,FL,2015-06-05,2015-06,1985-04-11,34,Friday,ALCOHOL PROHIBITION,DRUGS & ALCOHOL,STATUTORY CRIME,MIAMI,,,
2,16,"GAITOR, RASHAD ZAROSHUA",UNKNOWN,NaN,MIAMI,FL,2015-06-05,2015-06,1991-08-19,27,Friday,WARRANT,MISCELLANEOUS CRIMES,OTHER,MIAMI,,,
3,24,"WILLIAMS, ALAN J",UNKNOWN,NaN,MIAMI,FL,2015-06-03,2015-06,1964-04-28,55,Wednesday,ALCOHOL PROHIBITION,DRUGS & ALCOHOL,STATUTORY CRIME,MIAMI,,,
4,29,"WYNTER, KALONJI ABAYOMI",HOMELESS,NaN,HOMELESS,FL,2015-06-03,2015-06,1977-05-19,42,Wednesday,WARRANT,MISCELLANEOUS CRIMES,OTHER,HOMELESS,,,


In [11]:
base_url = "http://open.mapquestapi.com/geocoding/v1/" 
# key = "RAzgkAZaU07DwvFLcYg1cbTaTd5Wf3DQ"

# # use iterrows to iterate through pandas dataframe
for index, row in df.iterrows():

#     # get address form df
    address = str(row['Address'])
#     print(address)

#     # get city form df
    city = row['City']
#     print(city)
    
#     # assemble url and make API request
# #   http://open.mapquestapi.com/geocoding/v1/address?key=KEY&location=Washington,DC

    url = f"{base_url}address?key={key}&location={address},FL"
#     print(url)
    response = requests.get(url).json()
# #     print(response)
    
#     # extract results
    results = response['results']
# #     pprint(results)

    try:
        
        df.loc[index, 'Zip_code_from_api'] =  results[0]["locations"][0]["postalCode"]
        df.loc[index, 'lat'] = results[0]["locations"][0]["latLng"]['lat']
        df.loc[index, 'lon'] = results[0]["locations"][0]["latLng"]['lng']
        
    except (KeyError, IndexError):
        pass


In [13]:
df.head()

,Unnamed: 0,Defendant,Address,Zip,City,State,Booking_Date,Booking_year_month,Date_of_birth,Age,day_of_week,Crime_Family,Crime_Family1,Crime_Family2,CityRN,lat,lon,Zip_code_from_api,Zip_code
0,1,"DITAS, MICHAEL",7484 NW 169 TERR,NaN,MIAMI,FL,2015-06-05,2015-06,1982-02-17,37,Friday,"THEFT, ROBBERY, AND RELATED CRIMES",ASSAULT & BATTERY,PERSONAL CRIME,MIAMI,39.7837,-100.446,,
1,4,"DORSEY, JOHN",651 NW 177TH STREET 101,NaN,MIAMI GARDENS,FL,2015-06-05,2015-06,1985-04-11,34,Friday,ALCOHOL PROHIBITION,DRUGS & ALCOHOL,STATUTORY CRIME,MIAMI,25.9372,-80.2118,,33169
2,16,"GAITOR, RASHAD ZAROSHUA",UNKNOWN,NaN,MIAMI,FL,2015-06-05,2015-06,1991-08-19,27,Friday,WARRANT,MISCELLANEOUS CRIMES,OTHER,MIAMI,39.7837,-100.446,,
3,24,"WILLIAMS, ALAN J",UNKNOWN,NaN,MIAMI,FL,2015-06-03,2015-06,1964-04-28,55,Wednesday,ALCOHOL PROHIBITION,DRUGS & ALCOHOL,STATUTORY CRIME,MIAMI,39.7837,-100.446,,
4,29,"WYNTER, KALONJI ABAYOMI",HOMELESS,NaN,HOMELESS,FL,2015-06-03,2015-06,1977-05-19,42,Wednesday,WARRANT,MISCELLANEOUS CRIMES,OTHER,HOMELESS,39.7837,-100.446,,


In [15]:
df.to_csv("Resources/latlon.csv",index=True,header=True)